In [ ]:
MultiWorkerMetaOptimizer(
    my_active_learner,
    search_space,
    n_iter=300,
    n_jobs=1,
    memory="share",
    n_processes=1,
    n_workers=2,
)

* "sklearn-type API" for shared memory algo
* resume experiments, serialization
* "point that blows SBs mind"
* relation to active learning

In [ ]:
CachedExperiment(base_experiment)

# + man muss darauf achten, dass Instanzen nicht geklont werden?
# idea: multiton or singleton pattern to store the memory
# multiton based on base_experiment (same for clones)

optimizer1 = GridSearch(
    grid_params = [[1, 2, 3], [4, 5, 6]],
)
optimizer2 = MountEverestClimbing(
    distribution_space = LaplaceDistribution(
        loc=0,
        scale=1,
    ),
)

optimizers = [optimizer1, optimizer2]

cached_experiment = CachedExperiment(MyExperiment)

# or, we define optimizers without experiments first
for optimizer in optimizers:
    optimizer.set_params(experiment=cached_experiment)

# manual parallelization by user
parallelize(for optimizer in optimizers):
    optimizer.run()

# joblib parallelization
Parallel(n_jobs=2)(
    delayed(optimizer.run)()
    for optimizer in optimizers
)

class Scheduler:
    def __init__(self, backend, backend_params, experiment, cache=False):
        self.backend = backend
        self.backend_params = backend_params
        self.experiment = experiment
        self.optimizers = []
        if cache and not isinstance(experiment, CachedExperiment):
            self._experiment = CachedExperiment(experiment)
        else:
            self._experiment = experiment

    def add(self, optimizer, n_jobs):
        if self._experiment is not None:
            optimizer.set_params(experiment=self._experiment)
        self.optimizers.append((optimizer, n_jobs))

    # Question: max_time here or in init?
    def run(self, max_time=None):
        for optimizer, n_jobs in self.optimizers:
            Parallel(n_jobs=n_jobs)(
                delayed(optimizer.run)()
            )

sched = Scheduler(
    backend="joblib",
    backend_params = {"n_jobs": 2, "verbose": 10, "backend": "loky", "timeout": 10},
    experiment=experiment,  # optional, if done, then set_params and CachedExperiment is not necessary
)
# Question: Does experiment in init of scheduler override the experiment in the optimizer?
sched.add(optimizer1, n_jobs=2)
sched.add(optimizer2, n_jobs=2)
sched.run()

sched.results()

# returns df like in search_data, but wit additional column that is optimizer name

In [8]:
import pandas as pd

df = pd.DataFrame(
    {
        "optimizer": ["a", "b"],
        "n_jobs": [2, 2],
        "backend": ["loky", "loky"],
        "timeout": [10, 10],
    }
)
df


,optimizer,n_jobs,backend,timeout
0,a,2,loky,10
1,b,2,loky,10


In [ ]:
# option 1
cached_experiment = my_experiment.set_config("cache", True)

# option 2
from hyperactive import Cached
cached_experiment = Cached(my_experiment)

# option 3
cached_experiment = MyExperiment(stuf, stuff2, cache=True)

# option 4
experiment = MyExperiment(stuf, stuff2)
experiment.score(params, cache=True)